# Helper functions

In [1]:
model_id = "vinai/bartpho-syllable"

# Start

In [2]:
from transformers import MBartForCausalLM
import torch
import transformers
from transformers import AutoModel, AutoTokenizer

bartpho = MBartForCausalLM.from_pretrained(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_id)

c:\Users\vgtgd\miniconda3\envs\Vin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files='data/Vinmec/file.csv')

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 5610
    })
})

In [5]:
datasets["train"][10]["Answer"]

'Được giải đáp bởi Thạc sĩ, Bác sĩ Nguyễn Vĩnh Toàn - Khoa Liên Chuyên Khoa, Bệnh viện Đa khoa Quốc tế Vinmec Times City.Chào bạn,Với câu hỏi “Mũi họng có đờm sau sinh nên dùng thuốc gì?”, bác sĩ xin giải đáp như sau:Với các triệu chứng bạn mô tả thường là viêm mũi họng thông thường, có thể do thay đổi thời tiết, độ ẩm hoặc do nhiễm virus, không nên lạm dụng kháng sinh với trẻ 4 tháng tuổi. Bạn nên sử dụng nước muối ấm nhỏ mũi cho trẻ, có thể giảm ho bằng các thuốc ho thảo dược như Prospan. Tình trạng không cải thiện sau 1 tuần hoặc nếu ho tăng lên, sốt bạn nên đem trẻ đi khám để bác sĩ có phác đồ điều trị thích hợp.Nếu bạn còn thắc mắc về mũi họng có đờm sau sinh, bạn có thể đến bệnh viện thuộc Hệ thống Y tế Vinmec để kiểm tra và tư vấn thêm. Cảm ơn bạn đã tin tưởng và gửi câu hỏi đến Vinmec. Chúc bạn có thật nhiều sức khỏe.Trân trọng!'

In [6]:
count = 0
for i, example in enumerate(datasets["train"]):
    # print(example["Answer"])
    if example["Answer"] == None:
        count = count + 1
print(count)

226


In [7]:
def tokenize_function(examples):
    if examples["Answer"]:
        return tokenizer(examples["Question"]+examples["Answer"], truncation=True, padding='max_length', max_length=512)
    else:
        return tokenizer(examples["Question"], truncation=True, padding='max_length', max_length=512)
tokenized_datasets = datasets.map(tokenize_function, batched=False, remove_columns=datasets["train"].column_names)

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5610
    })
})

In [10]:
tokenized_datasets.keys()

dict_keys(['train'])

In [11]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_datasets.map(group_texts, batched=True)

Map:   0%|          | 0/5610 [00:00<?, ? examples/s]

Map: 100%|██████████| 5610/5610 [00:16<00:00, 342.51 examples/s]


In [12]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
from transformers import Trainer, TrainingArguments

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\vgtgd\miniconda3\envs\Vin\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [13]:
import evaluate


def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may contain extra tensors,
        # like past_key_values, but logits always come first
        logits = logits[0]
    return logits.argmax(dim=-1)

metric = evaluate.load("accuracy")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics but we need to shift the labels
    labels = labels[:, 1:].reshape(-1)
    preds = preds[:, :-1].reshape(-1)
    return metric.compute(predictions=preds, references=labels)

In [15]:
training_args = TrainingArguments(
    output_dir="check_point",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    # push_to_hub=True,
)

trainer = Trainer(
    model=bartpho,
    args=training_args,
    train_dataset=lm_dataset["train"],
    # eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

c:\Users\vgtgd\miniconda3\envs\Vin\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
prompt = "Mũi họng có đờm sau sinh nên dùng thuốc gì?"

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=bartpho)
generator(prompt)